In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
df=pd.read_csv("/DATA/jimson/Rishi/Priyadrasta/AutoMM/augmented_1st_Floor_Dataset_rescored_new.csv")
df.head(5)

In [ ]:
# df['key']=df['Var1'].apply(lambda x: int(x.split("_")[2]))
# len(df['key'].unique())

In [ ]:
df.shape

In [ ]:
df['label'].value_counts()

In [ ]:
df['label']=df['label'].apply(lambda x: int(x>0))
df['label'].value_counts()

In [ ]:
df.columns

In [ ]:
df=df[['image','label']]

In [ ]:
# df['label'].replace({1:'N',0:'NN'},inplace=True)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.head(10)

In [ ]:
# /DATA/jimson/Rishi/Priyadrasta/papercode/first floor/Augmented_1st_Floor_AutoMM.ipynb

# from PIL import Image
# example_image = df.iloc[4045]['image']
# print(example_image)

# pil_img = Image.open(example_image)
# display(pil_img)

In [ ]:
split_loc = 5184
df.iloc[split_loc]

In [ ]:
from sklearn.model_selection import train_test_split
train_dataset=pd.read_csv("1traind.csv")
test_dataset = pd.read_csv("1testd.csv")
# train_dataset,test_dataset=df.iloc[:split_loc,:],df.iloc[split_loc:,:]
# train_dataset,test_dataset=train_test_split(df, test_size=0.3)
train_dataset.shape,test_dataset.shape

In [ ]:
# test_dataset['key']=test_dataset['Var1'].apply(lambda x: int(x.split("_")[2]))
# len(test_dataset['key'].unique())

In [ ]:

train_dataset.tail(5)

In [ ]:
# train_dataset.to_csv("1traind.csv",index=False)

In [ ]:
test_dataset.head(5)

In [ ]:
print(train_dataset['label'].value_counts(),test_dataset['label'].value_counts(),sep="\n")

In [ ]:
from autogluon.multimodal import MultiModalPredictor

In [ ]:
predictor = MultiModalPredictor.load('/DATA/jimson/Rishi/Priyadrasta/AutoMM/AutogluonModels/ag-20230114_165901')
# predictor = MultiModalPredictor(label="label")
# predictor.fit(
#     train_data=train_dataset,
#     time_limit=3600, # seconds
#     hyperparameters={"model.names":  ['timm_image'],"env.num_gpus": 1,"optimization.max_epochs":30,"optimization.patience": 20,
#     "optimization.optim_type": "adam","model.timm_image.checkpoint_name":"mobilenetv2_050"}
# )

In [ ]:
# fit_result = predictor.fit_summary()
# print('Top-1 train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

In [ ]:
# test_acc = predictor.evaluate(test_dataset)
# print('Top-1 test acc: %.3f' % test_acc['top1'])

In [ ]:
# prediction = predictor.predict(test_dataset)
# prediction.value_counts()

In [ ]:
scores = predictor.evaluate(test_dataset, metrics=["accuracy"])
print('Top-1 test acc: %.3f' % scores["accuracy"])

In [ ]:
roc_auc = predictor.evaluate(test_dataset)
# print('Top-1 test acc: %.3f' % scores["roc_auc"])
roc_auc

In [ ]:
predictions = predictor.predict(test_dataset)
predictions.value_counts()

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix
conf_matrix= confusion_matrix(test_dataset['label'].to_numpy(),predictions.to_numpy())
conf_matrix

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
 
fig, ax = plot_confusion_matrix(conf_mat=conf_matrix, figsize=(4, 4), cmap=plt.cm.Blues)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix) 
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)
print("class-wise  Accuracy :",ACC)
print("class-wise  Precision :",PPV)
print("class-wise  Sensitivity :",TPR)
print("class-wise  Specificity :",TNR)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_dataset['label'].to_numpy(), predictions.to_numpy()))